## Hello, human.  It's our last week of python!  (Kind of...)

### I) Thoughtful reflection

Okay, let's take a minute to reflect on what we've learned recently.  Deep breaths, people.

Much of what one does with scientific computing is based on the small pieces that you already know!  We simply combine these in more complicated ways to generate more complex behavior!

This week, we'll study two indispensible tools for scientific computation: numerical integration and fitting.

* * *

### II) Numerical integration

Sometimes in the physical sciences, we need to integrate a function (or accumulate something *like* a function).  I've already shown you how Mathemagica can do this *symbolically*, but sometimes the function to be integrated is too complicated to integrate symbolically, OR we don't really care what the functional form of the integral is (we just want a number).  This might sound really abstract, but if you've ever played (almost) any video game or watched any CGI film, the action that you've seen is based on numercal solutions of integration of differential equations, which is basically just another way to think about integrating.

For our first introduction, let's say that we want to integrate a weirdo function like sinh(x). In order to visualize, we'll take it all the way back to the "rectangle method" from good ol' Calc I.  Recall images like this:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xmin = 0.0
xmax = 3.2
dx = 0.1

xlist = []
ylist = []
x_val = xmin
while x_val <= xmax:
    xlist.append(x_val)
    ylist.append(np.sinh(x_val))
    x_val += dx

xarray = np.array(xlist)
yarray = np.array(ylist)

#print(xarray)
#print(yarray)

x = np.linspace(0.0,4.0,num=10)
y = np.sinh(x)
fig, ax = plt.subplots()
ax.plot(xarray, yarray,'-')
ax.bar(x-0.05,y, color = 'orange')
plt.xlim(xmin, 3.0)
plt.ylim(-0.1,10)
plt.ylabel("sinh[x]")
plt.xlabel("x")

#ax.plot(x, 1/x)
#ax.plot(x, np.log(x))
#ax.set_aspect('equal')
ax.grid(True, which='both')

ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')

plt.show()

Maybe you're having unpleasant flashbacks right now. BUT! All that this is meant to show is that we often begin talking about integration as the addition of the vertical bars that approximate the area under a function:
$$ \textrm{area} \approx \displaystyle\sum_{i=1}^{N} f(x_i) \cdot \Delta x$$
This just means that we sum up the areas of some rectangles whose widths are $\Delta x$ and whose heights are the value of the function $f(x)$ at some value of $x$ (usually either the lft or right side of a rectangle).
$N$ is the number of bars that we consider, so if we're "integrating" between $x_0$ and $x_f$,
$$\Delta x = \frac{|x_f - x_0|}{N}$$
In this example, the bars have a width 
$$\Delta x = 0.4$$
that is large compared the rate at which the function changed, and the result is that the approximation of the integral of sinh(x) with *these* bars would be bad -- we'd overshoot the value of the integral by quite a bit.

When we go from an approximating sum to an integral, we shrink the increment in x to an infinitesimal:
$$ \textrm{area} = \displaystyle\int_{x_0}^{x_f} f(x) dx \equiv \lim_{\Delta x \rightarrow 0} \displaystyle \sum_{i=1}^{N} f(x_i)\cdot \Delta x = \lim_{N \rightarrow \infty} \displaystyle \sum_{i=1}^{N} f(x_i)\cdot \frac{x_f - x_0}{N}$$


Computers, being digital, are really bad at going to the "continuum limit", $N\rightarrow \infty$.  BUT we can often make $N$ really big (but less than $\infty$) or $dx$ really small (but bigger than 0), and get away with a decent approximation of the function.

****

####Example numerical integral!

Let's integrate the function 
$$v(t) = -9.8t + 50.0 - 12.0t^2$$

from $t = 0$ to $t = 2.0$  We'll choose $dt=0.01$. (We'll later refer to $dt$ as the *timestep*.)

I'm only going to print out five digits of the integral, for reasons that we'll discuss later.


In [ ]:
def v(t):         # I'll define a function for the thing that I want to integrate
    return (-9.8)*t + 50.0 - 12.0*t*t

tmin = 0.0        # set up the integral parameters
tmax = 2.0
dt = 0.01

pos_sum = 0.0     # the variable that I'll add to each time through the loop

tarr, varr = [], []
t = tmin
while t <= tmax:
    tarr.append(t)
    varr.append(v(t))
    pos_sum += v(t)*dt
    t = t + dt
    
output_str = "%0.5f \t %0.5f" % (t,pos_sum)  # this is a fancy way to format numbers into strings.
                                             # try it out!
print(output_str)

tarr = np.array(tarr)
varr = np.array(varr)

plt.plot(tarr,varr,'r-')
plt.xlim(tmin, tmax)
plt.ylabel("v[t] (m/s)")
plt.xlabel("t (s)")
plt.show()

Note that in this case, I'm essentially ealuating the function $v(t)$ at the left/lower boundary of each rectangle.  So, our approximation should be slightly below the actual value of the integral.  This seems to be making sense.
****
Let's investigate how the value of the integral depends on our $dx$.  As $dx$ gets smaller, our estimate should approach the actual value of the integral.  Check this (might take a minute to run!):

In [ ]:
def integrate_v(deltat):
    mint = 0.0
    maxt = 2.0
    integ = 0.0
    t = mint
    while t <= maxt:
        integ += v(t)*deltat
        t = t + deltat
    return integ

dts = [1.0, 0.1, 0.01, 0.001, 1e-4, 1e-5, 1e-6, 1e-7]

for dtval in dts:
    print(str(dtval) + "\t" + str(integrate_v(dtval)))

It appears to me that the true value of the integral is 48.4.  (You should do it by hand to determine what the actual number is.)  
When we set $dt$ to the smallest value (1e-7 means $1\times 10^{-7}$), we get a value that is within three millionths of the actual value. 
This discrepancy might be tolerable depending on the type of calculation one is performing.
****
For my last trick, let's take a look at the functional form of the integral.  Recall that the relationship between the integral function and the function being integrated is
$$ F(t) = \displaystyle\int_{0}^{t} f(t') dt'$$
(plus possibly some integration constant).

In order to display the functional form of the integral, we'll plot the value of the sum versus $t$ as we progress through our loop.  In this case, I'll integrate a more common function: $\cos(u)$.  I'll choose $du = 0.0001$, but only plot the values every 100 steps through the calculation.

In [ ]:
ulis = []
cos_vals = []
cos_int = []

int_sum = 0.0
umin = 0.0
umax = 10.0
du = 0.0001
u_val = umin

nloops = 0

while u_val <= umax:
    int_sum += np.cos(u_val)*du
    if nloops % 100 == 0:
        ulis.append(u_val)
        cos_vals.append(np.cos(u_val))
        cos_int.append(int_sum)
    u_val += du
    nloops += 1

ulis = np.array(ulis)
cos_vals = np.array(cos_vals)
cos_int = np.array(cos_int)

fig, ax = plt.subplots()
ax.plot(ulis,cos_vals,'b-')
ax.plot(ulis,cos_int,'r-')
plt.ylabel("cos[u]")
plt.xlabel("u")

plt.show()


WAIT!  I have seen this somewhere before... but where... could it be SIN(X)?  Yes, yes it could be.

### Later...

Later when we get into Differential Equations, we'll find out some of the real utility of integration techniques.  But we need to let our math skills catch up to our programming skills.

****

### III) VPython

We'll now begin the fun stuff! Numerical integration allows us to simulate the motion of objects under simple, or even not-so-simple, forces!  This begins with Newton's Second Law:  
$$ \displaystyle\sum\vec{F} = m \vec{a}, $$
Where $m$ is the particle's mass, $\vec{a}$ is its acceleration, and $\displaystyle\sum\vec{F}$ is the net force that it experiences.
(Does this look familiar?)

Lurking in NII is a differential equation, a subject that we'll get to later in the course.  Most of the problems that you solved with NII in PHY 107, 108, 101, and 102 considered objects that move in response to a constant force -- this allows us to use the kinematic equations for constant acceleration.  This is only a small slice of physical phenomena, however!  More interesting systems exhibit forces that are dependent on position, or time (, or both), or other factors that may vary as well.  For these cases, the acceleration will not be constant, and the kinematic equations do not work.

The way that we treat such systems computationally is to integrate numerically.  We begin with a particle that has initial values of position and velocity at a given time.  Let's consider a particle that moves in one dimension and specify the particle's position and velocity at time $t_0$.
$$x(t_0) = x_0, \; v(t_0) = v_0$$ 
Given these, we can calculate the force that acts on the particle at $t_0$, as well.  The force might have some nasty dependence on the initial parameter, so we'll just remain agnostic right now and call it $F_x(x,v,t)$.

We can calculate the acceleration that the particle experiences at $t_0$ with the functional form of $F$ using NII:
$$F(x_0,v_0,t_0) \implies a(t_0) = \frac{F(x_0,v_0,t_0)}{m}$$
Evolving the position forward in time then amounts to choosing a time step, $\Delta t$, and *numerically integrating* the velocity and position.  We generate the velocity at time $t_1 = t_0 + \Delta t$ as
$$v_1 \equiv v(t_0 + \Delta t) = a(t_0)\cdot\Delta t + v(t_0) = \frac{F(x_0,v_0,t_0)}{m} \Delta t + v_0$$
and the position at the next time as
$$x_1 \equiv x(t_0 + \Delta t) = v(t_1)\cdot\Delta t + x(t_0)$$

This process repeats to get the position and velocity at following times $t_2$, $t_3$, ...


VPython is a modulde that allows for easy visualization of physical systems.  The block below codes up the NII algorithm described above and generates an image that gets updated in real time!  We'll see how this works in more detail in the future, but for now, you might want to goof around with the bouncing ball demo.  Check out the line that calculates the force, and see if you can figure out what it's doing.

In [ ]:
from vpython import *

import random as rand

scene.caption = """Right button drag or Ctrl-drag to rotate "camera" to view scene.
To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
     On a two-button mouse, middle is left + right.
Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""

side = 4.0
thk = 0.3
s2 = 2*side - thk
s3 = 2*side + thk
wallR = box (pos=vector( side, 0, 0), size=vector(thk, s2, s3),  color = color.red)
wallL = box (pos=vector(-side, 0, 0), size=vector(thk, s2, s3),  color = color.red)
wallB = box (pos=vector(0, -side, 0), size=vector(s3, thk, s3),  color = color.blue)
wallT = box (pos=vector(0,  side, 0), size=vector(s3, thk, s3),  color = color.blue)
wallBK = box(pos=vector(0, 0, -side), size=vector(s2, s2, thk), color = color.gray(0.7))

ball = sphere (color = color.green, radius = 0.4, make_trail=True, retain=100, 
               trail_color=color.cyan)
ball.mass = 1.0
ball.velocity = vector(-2.0, 0.0, -2.9)
side = side - thk*0.5 - ball.radius
ball.pos = vector(side-0.3, side-0.3, side-0.3)

dt = 0.001
t = 0.0
f = vector(0, -9.8*ball.mass, 0.0)
a = (1/ball.mass)*f

while True:
    rate(400)
    t = t + dt
    ball.velocity = ball.velocity + a*dt 
    ball.pos = ball.pos + (ball.velocity/ball.mass)*dt
    if ball.pos.x < -side:
        ball.velocity.x = -ball.velocity.x
        ball.pos.x = -side
    if ball.pos.x > side:
        ball.velocity.x = -ball.velocity.x
        ball.pos.x = side
    if ball.pos.y < -side:
        ball.velocity.y = -ball.velocity.y
        ball.pos.y = -side
    if ball.pos.y > side:
        ball.velocity.y = -ball.velocity.y
        ball.pos.y = side
    if ball.pos.z < -side:
        ball.velocity.z = -ball.velocity.z
        ball.pos.z = -side
    if ball.pos.z > side:
        ball.velocity.z = -ball.velocity.z
        ball.pos.z = side
        



### Good luck, earthlings!

#### Sincerely,
The Automator
